In [2]:
"|IMPORT PACKAGES|"
import pandas             as pd
import datetime
from   bokeh.plotting     import show, figure, output_file, save
from   bokeh.io           import show, output_notebook, curdoc, export_png
from   bokeh.models       import ColumnDataSource,LinearAxis, Range1d, NumeralTickFormatter, LabelSet, Label, BoxAnnotation, DatetimeTickFormatter, Legend, LegendItem
from   bokeh.models.tools import HoverTool
from   bokeh.models       import Arrow, NormalHead, OpenHead, VeeHead
from   datetime           import datetime as dt

In [6]:
"|IMPORT DATA|"
path = r'https://github.com/ncachanosky/research/blob/master/Economic%20Series/'
file = r'Resumen%20Estadistico%20-%20Argentina.xlsx?raw=true'
IO   = path + file

sheet = 'MONTHLY'

data = pd.read_excel(IO, sheet_name = sheet, skiprows=2,usecols="B,JB,JC",engine='openpyxl')
data = data.dropna()
data = data.rename(columns={"Asalariados privados":"ASAL_PRI"})
data = data.rename(columns={"Asalariados públicos":"ASAL_PUB"})

In [7]:
data

,MONTH,ASAL_PRI,ASAL_PUB
288,2012-01-31,6068.155,2548.429
289,2012-02-29,6062.512,2607.158
290,2012-03-31,6072.122,2620.057
291,2012-04-30,6055.222,2623.593
292,2012-05-31,6065.645,2596.534
...,...,...,...
392,2020-09-30,5802.225,3215.571
393,2020-10-31,5798.660,3212.515
394,2020-11-30,5804.085,3230.865
395,2020-12-31,5802.666,3229.722


In [8]:
cds = ColumnDataSource(data)

#BUILD FIGURE
p = figure(title = "EL HUB ECONÓMICO | ASALARIADOS SECTOR PÚBLICO Y PRIVADO (EN MILES)",
           plot_height  = 400,
           plot_width   = 700,
           x_axis_type  = "datetime",
           y_range      = (5500, 6500),
           y_axis_label = 'Asalariados privados')

p.toolbar_location = 'right'
p.toolbar.autohide = True

#AXIS 1 (LEFT)
p1 = p.line("MONTH", "ASAL_PRI", color="green", alpha=0.7, width=1,legend_label='Asalariados privados',source=cds)
p.add_tools(HoverTool(renderers=[p1], tooltips = [("privados", "@ASAL_PRI{0,0}")], mode="vline"))

#AXIS 2 (RIGHT)
p.extra_y_ranges = {"y2": Range1d(start=2500, end=3500)}
p.add_layout(LinearAxis(y_range_name = "y2", axis_label = "Asalariados públicos"), 'right')
p2 = p.line("MONTH", "ASAL_PUB", color="red"  , alpha=0.7, width=1,legend_label='Asalariados públicos', y_range_name = "y2", source=cds)
p.add_tools(HoverTool(renderers=[p2], tooltips = [("públicos", "@ASAL_PUB{0,0}")], mode="vline"))

#ADD NOTES
#Presidents' shades and names
CFK       = BoxAnnotation(left=dt(2007,12,10).timestamp()*1000, right=dt(2015,12,10).timestamp()*1000, line_color="black" ,fill_color="blue"  , fill_alpha=0.1)
Macri     = BoxAnnotation(left=dt(2015,12,10).timestamp()*1000, right=dt(2019,12,10).timestamp()*1000, line_color="black", fill_color="yellow", fill_alpha=0.1)
Fernandez = BoxAnnotation(left=dt(2019,12,10).timestamp()*1000, right=dt(2021, 4,30).timestamp()*1000, line_color="black", fill_color="blue"  , fill_alpha=0.1)

x1 = (dt(2012, 1,31).timestamp()*1000 + dt(2015,12,10).timestamp()*1000) / 2
x2 = (dt(2015,12,10).timestamp()*1000 + dt(2019,12,10).timestamp()*1000) / 2
x3 = (dt(2019,12,10).timestamp()*1000 + dt(2021, 4,30).timestamp()*1000) / 2

CFK_N       = Label(x=x1, y=5550, text="C. Kirchner", text_font_size="10pt", text_align="center", text_baseline="middle", angle_units="deg", angle= 0)
Macri_N     = Label(x=x2, y=5550, text="Macri"      , text_font_size="10pt", text_align="center", text_baseline="middle", angle_units="deg", angle= 0)
Fernandez_N = Label(x=x3, y=5550, text="AF"         , text_font_size="10pt", text_align="center", text_baseline="middle", angle_units="deg", angle= 0)

p.add_layout(CFK)
p.add_layout(Macri)
p.add_layout(Fernandez)

p.add_layout(CFK_N)
p.add_layout(Macri_N)
p.add_layout(Fernandez_N)

#Arrows
crisis_fx_text  = Label(x = dt(2018, 4, 8).timestamp()*1000,
                        y = 6400                           ,
                        text           = 'Crisis cambiaria',
                        text_font_size = '12px'            ,
                        text_color     = 'green'           ,
                        text_align     = 'center'          ,
                        text_baseline  = 'bottom'
                        )

crisis_fx_arrow = Arrow(end=VeeHead(line_color='green', line_width=2, line_alpha=0.5, size=10, fill_color='green', fill_alpha=0.75),
                        line_color='green',
                        x_start = dt(2018, 4, 1).timestamp()*1000,
                        x_end   = dt(2018, 4, 1).timestamp()*1000,
                        y_start = 6400,
                        y_end   = 6300
                        )

p.add_layout(crisis_fx_text)
p.add_layout(crisis_fx_arrow)

#LEGEND
p.legend.location     = 'top_left'
p.legend.orientation  = 'vertical'
p.legend.click_policy = 'mute'
output_notebook()
show(p)

Loading BokehJS ...

In [ ]:
"|EXPORT .PNG FILE|"

export_png(p, filename="asalariados.png")

In [ ]:
"|CREATE HTML FILE|"

output_file(filename="asalariados.html", title="Asalariados público y privado")
save(p)

In [9]:
"|CREATE JSON FILE|"
import json
import bokeh.embed

from bokeh.embed import json_item

j = json.dumps(json_item(p, "asalariados"))
with open("04.03_asalariados.json", "w") as fp:
    json.dump(j, fp)